In [1]:
import json
import os
from datasets import load_dataset


ds = load_dataset("PrimeIntellect/verifiable-coding-problems", split="train", trust_remote_code=True)

print(ds[0].keys())

/opt/conda/envs/rllm-xiao/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


dict_keys(['source', 'task_type', 'in_source_id', 'prompt', 'gold_standard_solution', 'verification_info', 'metadata', 'problem_id'])


In [3]:
import ast
dataset = []
for entry in ds:
    gold_standard_solution = entry["gold_standard_solution"]
    if gold_standard_solution is None:
        continue
    if gold_standard_solution.startswith("```python") and gold_standard_solution.endswith("```"):
        tests = entry["verification_info"]
        if isinstance(tests, str):
            try:
                tests = ast.literal_eval(tests)
            except (ValueError, SyntaxError) as e:
                #Try Json loads instread
                try: 
                    tests = json.loads(entry["verification_info"])
                except (json.JSONDecodeError, SyntaxError, ValueError) as e:
                    print(repr(entry["verification_info"]))
                    print(f"Error in json.loads: {e}")
                    continue
        assert isinstance(tests, dict), "Tests should be a dictionary"
        tests = tests["test_cases"]
        if len(tests) == 0:
            continue 
        new_entry = {
            "problem": entry["prompt"],
            "solutions": [gold_standard_solution],
            "tests":tests,
        }

        #Assert tests is a dictionary 
        assert isinstance(tests, list), "Tests should be a dictionary"
        assert "input" in tests[0], "Tests should have an input key"
        assert "output" in tests[0], "Tests should have an output key"

        dataset.append(new_entry)

print(len(dataset))
print(dataset[0])
dataset = dataset
output_dir = os.path.abspath("../../train/code")
output_file = os.path.join(output_dir, "verify.json")
with open(output_file, "w") as f:
    json.dump(dataset, f, indent=4)

34755
{'problem': 'Solve the following coding problem using the programming language python:\n\nPolycarp has $n$ different binary words. A word called binary if it contains only characters \'0\' and \'1\'. For example, these words are binary: "0001", "11", "0" and "0011100".\n\nPolycarp wants to offer his set of $n$ binary words to play a game "words". In this game, players name words and each next word (starting from the second) must start with the last character of the previous word. The first word can be any. For example, these sequence of words can be named during the game: "0101", "1", "10", "00", "00001".\n\nWord reversal is the operation of reversing the order of the characters. For example, the word "0111" after the reversal becomes "1110", the word "11010" after the reversal becomes "01011".\n\nProbably, Polycarp has such a set of words that there is no way to put them in the order correspondent to the game rules. In this situation, he wants to reverse some words from his set 